In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Model#type:ignore
from tensorflow.keras.layers import (#type:ignore
    Input, Conv3D, Conv2D, Reshape, MaxPooling3D, MaxPooling2D,
    Flatten, Dense, Dropout, BatchNormalization
) 
from tensorflow.keras.optimizers import Adam#type:ignore
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint#type:ignore
from tensorflow.keras.utils import to_categorical#type:ignore
np.random.seed(42)
tf.random.set_seed(42)
%matplotlib inline
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 8)

In [5]:
X_train = np.load('../data/processed/X_train_pca.npy')
X_val = np.load('../data/processed/X_val_pca.npy')
X_test = np.load('../data/processed/X_test_pca.npy')
y_train = np.load('../data/processed/y_train.npy')
y_val = np.load('../data/processed/y_val.npy')
y_test = np.load('../data/processed/y_test.npy')
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Training labels shape: {y_train.shape}")

Training data shape: (6149, 25, 25, 30, 1)
Validation data shape: (2050, 25, 25, 30, 1)
Test data shape: (2050, 25, 25, 30, 1)
Training labels shape: (6149,)


In [6]:
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))

print("Class weights:")
for cls, weight in class_weights.items():
    print(f"Class {cls}: {weight:.2f}")

Class weights:
Class 0: 13.73
Class 1: 0.45
Class 2: 0.77
Class 3: 2.69
Class 4: 1.33
Class 5: 0.88
Class 6: 22.61
Class 7: 1.34
Class 8: 32.03
Class 9: 0.66
Class 10: 0.26
Class 11: 1.08
Class 12: 3.12
Class 13: 0.51
Class 14: 1.66
Class 15: 6.86


In [7]:
def create_hybridsn_model(input_shape, num_classes):
    input_layer = Input(shape=input_shape)
    x = Conv3D(filters=8, kernel_size=(3, 3, 7), activation='relu')(input_layer)
    x = BatchNormalization()(x)
    x = Conv3D(filters=16, kernel_size=(3, 3, 5), activation='relu')(x)
    x = BatchNormalization()(x)
    shape = x.shape
    x = Reshape((shape[1], shape[2], shape[3] * shape[4]))(x)
    x = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.4)(x)
    output_layer = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    return model
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3], 1)
num_classes = len(np.unique(y_train))
model = create_hybridsn_model(input_shape, num_classes)
model.summary()

E0000 00:00:1757450681.780205   11650 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1757450681.787293   11650 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 25, 25, 30, 1)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d (Conv3D)                 │ (None, 23, 23, 24, 8)  │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 23, 23, 24, 8)  │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 21, 21, 20, 16) │         5,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 21, 21, 20, 16) │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 21, 21, 320)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 19, 19, 32)     │        92,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 19, 19, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 17, 17, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 17, 17, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 18496)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     4,735,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │         2,064 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,887,648 (18.64 MB)

 Trainable params: 4,887,408 (18.64 MB)

 Non-trainable params: 240 (960.00 B)